In [3]:
import pandas as pd

# https://portal.gdc.cancer.gov/
# tar -xvzf C:\Users\Ameno\Downloads\clinical.cases_selection.2022-03-29.tar.gz -C C:\Users\Ameno\Downloads\HNSC_clinical

In [4]:
# df = pd.read_csv('sample_metadata.txt', delimiter = "\t")
g = pd.read_csv('Data/HNSC/tcga_scaled01_rnaseq.tsv', delimiter = "\t", index_col=0)
# b.head()
# b.head()
# df.head()
# list(df.columns)

In [5]:
# Explore SNF data
g_snf = pd.read_csv("Data\COAD\COLON_Gene_Expression.txt", delimiter = "\t", index_col=0).T
print(g_snf.head(2))
# print(g.head(2))

                                 TRHDE     RPE65     OR4N5    TRIM60  \
                             -0.042882  0.583696  0.369836  0.985446   
TCGA-A6-2670-01A-02R-0821-07 -0.231989 -1.779292  0.913945 -0.452253   

                                 LRIT2    ZNF677  C1orf129  MGC32805  \
                             -1.578451 -0.503370  0.112099  0.818456   
TCGA-A6-2670-01A-02R-0821-07 -0.049176 -1.234968 -0.988846 -1.521509   

                                  HBG2   TMEM163  ...    FBXO10  \
                             -1.049611 -0.304902  ... -0.147438   
TCGA-A6-2670-01A-02R-0821-07  0.511784 -0.062709  ...  0.390474   

                              RP11-218C14.6    TEX261     OBFC1    CHRDL2  \
                                  -0.845946  2.100277 -0.304646  1.097201   
TCGA-A6-2670-01A-02R-0821-07       0.125562 -0.587602 -0.309476 -0.420616   

                              ATP6V1G3    HOTAIR     ALG10    KCNMB4       TK2  
                              0.802950 -0.513098  0

In [30]:
x_cancer_type = ["COAD", "ESCA", "HNSC", "READ", "STAD"]

#stad has the most normal samples 
cancer = x_cancer_type[0]
# cancer = "HNSC"
# cancer = "ESCA"
# cancer = "READ"

b = pd.read_csv(f'Data/{cancer}/Microbial/bacteria.sample.relabund.phylum.txt', delimiter = "\t", index_col=0).T

In [31]:
g.head()
# b.head()

g.iloc[:,3]
g_samples = list(g.index.values)
b_samples = list(b.index.values)
set(g_samples).intersection(b_samples)
b_samples
# print(g.columns)
# print(g.describe())

find = "TCGA-CR-6491"
# find = "TCGA-CV-7090"

# TCGA-ZX-AA5X-01 does not occur, and has duplicates. Not true, it has it. It just doesn't have some of the elevens ( other elevens it does ), and take care of the b ones

# HNSC 

def findOverlap(a_samples, b_samples, ignoreVial=False):
    overlapping_count = 0
    normal_overlapping_count = 0
    missing = []
    control_count = 0
    for find in a_samples:
        occurrences = 0
        for s in b_samples:
            if ignoreVial and find[-1]=="A":
                find = find[:-1]
            if find in s:
                occurrences+=1
        if occurrences<1:
            # print(find, " does not have any occurrences")
            missing.append(find)
        else:
            overlapping_count += 1
        if ((ignoreVial and find[-2] == "1") or (not ignoreVial and find[-3] == "1")):
            control_count += 1
            if occurrences > 0:
                normal_overlapping_count += 1
    print("no occurrences:", missing)
    print("overlapping count: ", overlapping_count, "/", len(a_samples))
    print("normal count: ", control_count, "/", len(a_samples))
    print("normal overlapping count: ", normal_overlapping_count, "/", control_count)
    return missing
rna_stad_missing = findOverlap(b_samples, g_samples, ignoreVial=True)
l1=[]
for i in b:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')


# tumor normal and control 
import re
for s in g_samples:
    if "4183" in s:
    # if re.search(r"[1].$",s):
    # if "-21" in s:
        
        print(s)

no occurrences: ['TCGA-AA-A010-01', 'TCGA-AA-A00Z-01', 'TCGA-A6-2676-01', 'TCGA-A6-5659-01B', 'TCGA-AA-3666-01', 'TCGA-AA-A01V-11', 'TCGA-AA-3947-01', 'TCGA-A6-2679-01', 'TCGA-AA-3558-01', 'TCGA-AA-A01V-01', 'TCGA-AY-4070-01', 'TCGA-AA-3494-11', 'TCGA-A6-2680-01', 'TCGA-A6-2677-01B', 'TCGA-AA-3529-01', 'TCGA-A6-3810-11', 'TCGA-AA-3681-01', 'TCGA-CM-4750-01', 'TCGA-A6-3808-11', 'TCGA-AA-3664-01', 'TCGA-AA-A02W-01', 'TCGA-AA-3516-01', 'TCGA-AA-A01Q-01', 'TCGA-AA-3549-01', 'TCGA-AA-A01S-01', 'TCGA-AA-3692-01', 'TCGA-AA-3510-11', 'TCGA-AA-A01K-01', 'TCGA-AA-A01D-01', 'TCGA-AA-3494-01', 'TCGA-AA-3812-01', 'TCGA-A6-6780-01B', 'TCGA-AA-3956-01', 'TCGA-AA-A01T-01', 'TCGA-AZ-4308-01', 'TCGA-AA-A01R-11', 'TCGA-AA-3510-01', 'TCGA-AA-3861-01', 'TCGA-A6-3807-01', 'TCGA-A6-2671-01', 'TCGA-AZ-4681-01', 'TCGA-AA-3514-01', 'TCGA-AA-3495-11', 'TCGA-AA-A02O-01', 'TCGA-AA-A02R-01', 'TCGA-A6-5656-01B', 'TCGA-CM-4752-01', 'TCGA-AA-3553-01', 'TCGA-AA-A03F-11', 'TCGA-A6-6650-01B', 'TCGA-AA-A01C-01', 'TCGA-AA-

In [32]:
x_type = ["cnv", "methylation", "rnaseq"]
# x_cancer_type = ["COAD", "READ", "STAD"]
xe_rna_stad_samples = pd.read_csv(rf"Data\{cancer}\xenos_{x_type[2]}_samples.txt", names=["sample"], header=None)
xe_rna_stad_samples.head(2)
xe_rna_stad_missing = findOverlap(b_samples, xe_rna_stad_samples["sample"].tolist())
xe_rna_stad_missing = map(lambda x: x[:-1], xe_rna_stad_missing)
with_xe_difference = set(xe_rna_stad_missing).difference(set(rna_stad_missing))
from_xe_difference = set(rna_stad_missing).difference(set(xe_rna_stad_missing))

print("x-tc:", with_xe_difference, " tc-x:", from_xe_difference)
# findOverlap(b_samples, g_snf_samples_mapped)

no occurrences: ['TCGA-AA-A01V-11A', 'TCGA-AA-3558-01A', 'TCGA-AA-3494-11A', 'TCGA-A6-3810-11A', 'TCGA-A6-3808-11A', 'TCGA-AA-3510-11A', 'TCGA-AA-A01R-11A', 'TCGA-AZ-4681-01A', 'TCGA-AA-3495-11A', 'TCGA-AA-A03F-11A', 'TCGA-AA-3509-11A', 'TCGA-A6-3807-11A', 'TCGA-AA-A01P-11A', 'TCGA-AA-3502-11A', 'TCGA-AA-3529-11A', 'TCGA-AA-A01T-11A', 'TCGA-AA-A03J-11A', 'TCGA-AA-A02O-11A', 'TCGA-AA-A01X-11A', 'TCGA-AA-A01S-11A']
overlapping count:  126 / 146
normal count:  21 / 146
normal overlapping count:  3 / 21
x-tc: set()  tc-x: {'TCGA-AA-A01Q-01', 'TCGA-AA-3518-01', 'TCGA-AA-A01D-01', 'TCGA-A6-2683-01', 'TCGA-AA-3966-01', 'TCGA-AA-3968-01', 'TCGA-AA-A01I-01', 'TCGA-AA-3494-01', 'TCGA-CM-4752-01', 'TCGA-A6-2681-01', 'TCGA-A6-6780-01B', 'TCGA-A6-2676-01', 'TCGA-AA-A02O-01', 'TCGA-AA-3529-01', 'TCGA-AA-3494-11', 'TCGA-AA-3558-01', 'TCGA-A6-5656-01B', 'TCGA-A6-3809-01B', 'TCGA-AA-3516-01', 'TCGA-AA-3812-01', 'TCGA-AA-3861-01', 'TCGA-A6-3807-01', 'TCGA-AA-3529-11', 'TCGA-AA-3947-01', 'TCGA-A6-3808-11

In [14]:
g2 = pd.read_csv('Data\pancan_scaled_rnaseq.tsv\pancan_scaled_rnaseq.tsv', delimiter = "\t", index_col=0)

In [18]:
g2_samples = list(g2.index.values)
# len(g_samples)
same = set(g2_samples).intersection(g_samples)
len(same)

10459

In [12]:
# g.rename(columns={0:"name"}, inplace=True)
g.index.name = "name"
print(g.head(2))
b.index
b.head(2)

                   RPS4Y1      XIST      KRT5  AGR2  CEACAM5     KRT6A  \
name                                                                     
TCGA-02-0047-01  0.678296  0.289910  0.034230   0.0      0.0  0.084731   
TCGA-02-0055-01  0.200633  0.654917  0.181993   0.0      0.0  0.100606   

                    KRT14   CEACAM6     DDX3Y     KDM5D  ...   FAM129A  \
name                                                     ...             
TCGA-02-0047-01  0.031863  0.037709  0.746797  0.687833  ...  0.440610   
TCGA-02-0055-01  0.050011  0.092586  0.103725  0.140642  ...  0.620658   

                  C8orf48    CDK5R1    FAM81A  C13orf18     GDPD3     SMAGP  \
name                                                                          
TCGA-02-0047-01  0.428782  0.732819  0.634340  0.580662  0.294313  0.458134   
TCGA-02-0055-01  0.363207  0.592269  0.602755  0.610192  0.374569  0.722420   

                  C2orf85   POU5F1B     CHST2  
name                                     

name,Bacteroidetes,Proteobacteria,Firmicutes,Fusobacteria,Verrucomicrobia,Chloroflexi,Thermotogae,Actinobacteria,Spirochaetes,Synergistetes,Tenericutes
TCGA-AG-3999-01A,0.647603,0.000000,0.067323,0.276764,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-AG-A01L-01A,0.579041,0.234426,0.160304,0.013424,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
b.rename(index= lambda s: s[:-1] if s[-1]=="A"else s, inplace=True)
multi = b.join(g, how="inner")
# multi.loc["TCGA-CV-5970-11"]
multi.index.name = "patient"
# multi["patient"]
# multi["tumor"] = multi.apply(lambda row: 1 if re.search(r"[0].$",row.name) else 0)
multi["tumor"] = multi.apply(lambda row: 1 if re.search(r"[0].$",row.name) else 0, axis=1)
# multi.apply(lambda row: print(row.name))
multi.to_csv(f"Data/{cancer}/gene_biome_{cancer}.csv")
# pd.merge(b, g, how="inner", left_on="name")
# g.merge(b, how="inner", on="name")